# RAG over complex PDF


## Setting
 - Auto Reload
 - path for utils

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

In [ ]:
!pip install "unstructured[all-docs]"

## 1. Bedrock Client 생성

In [48]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [49]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v3-sonnet 모델 로딩

### LLM 로딩 (Claude-v3-sonnet)

In [56]:
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [55]:
llm_text = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Sonnet"),
    client=boto3_bedrock,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f098c286b90>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f098c25e560>])

### Embedding 모델 선택

In [82]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [83]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [84]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [85]:
import boto3
from utils.ssm import parameter_store

In [86]:
region=boto3.Session().region_name
pm = parameter_store(region)

INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [87]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [88]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [89]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

index_name: v01-genai-poc-parent-doc-retriever


### OpenSearch Client 생성

In [90]:
from utils.opensearch import opensearch_utils

In [91]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

BedrockChat with claude3: https://medium.com/@dminhk/building-with-anthropics-claude-3-on-amazon-bedrock-and-langchain-%EF%B8%8F-2b842f9c0ca8

In [73]:
from langchain_core.messages import HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [79]:
system_prompt = "You are an assistant tasked with summarizing text, table and image."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

In [80]:
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{docs[-5].metadata['image_base64']}",
        },
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{docs[-4].metadata['image_base64']}",
        },
    },
    {
        "type": "text",
        "text": "{question}"
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [78]:
prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

chain = prompt | llm_text | StrOutputParser()

response = chain.invoke({"question": "두 사진에 대해서 각각 설명하고 차이점을 알려줘"})

print(response)

이 두 이미지는 비슷한 그래프를 보여주지만, 약간의 차이점이 있습니다.

첫 번째 이미지는 한국어로 "수익률 그래프"라는 제목이 있습니다. 가로축은 "직조기준가격대비 기준자산가격(%)"으로 표시되어 있고, 세로축은 "세전수익률"로 나와 있습니다. 

두 번째 이미지에는 제목이 없지만, 축 레이블과 데이터 값은 동일합니다.

두 그래프 모두 원금보장 수준인 100%에서 시작하여 오른쪽 상단으로 기울어진 직선을 보여주고 있습니다. 이 직선은 기준자산가격이 130%일 때 상승폭여율이 약 70%임을 나타냅니다. 세전수익률은 21%로 표시되어 있습니다.

전반적으로 두 이미지는 동일한 정보를 전달하지만, 첫 번째 이미지에는 제목이 있는 반면 두 번째에는 제목이 없다는 점이 다릅니다.이 두 이미지는 비슷한 그래프를 보여주지만, 약간의 차이점이 있습니다.

첫 번째 이미지는 한국어로 "수익률 그래프"라는 제목이 있습니다. 가로축은 "직조기준가격대비 기준자산가격(%)"으로 표시되어 있고, 세로축은 "세전수익률"로 나와 있습니다. 

두 번째 이미지에는 제목이 없지만, 축 레이블과 데이터 값은 동일합니다.

두 그래프 모두 원금보장 수준인 100%에서 시작하여 오른쪽 상단으로 기울어진 직선을 보여주고 있습니다. 이 직선은 기준자산가격이 130%일 때 상승폭여율이 약 70%임을 나타냅니다. 세전수익률은 21%로 표시되어 있습니다.

전반적으로 두 이미지는 동일한 정보를 전달하지만, 첫 번째 이미지에는 제목이 있는 반면 두 번째에는 제목이 없다는 점이 다릅니다.


In [ ]:
prompt_text="""
\n\nHuman:
You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text.
Table or text chunk: {element}
\n\nAssistant:
"""

In [54]:
prompt1 = "나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."
messages = [
    HumanMessage(content=prompt1)
]

# messages = [
#     {"role": "user", "content": [{"type": "text", "text": prompt1}]},
# ]

response1 = llm_text.invoke(messages)

물론 생명보험과 손해보험의 주요 차이점을 설명해 드리겠습니다.

1. 보상대상
- 생명보험: 사람의 생명과 관련된 위험을 보장합니다. 예를 들어 사망, 상해, 질병 등의 위험을 담보합니다.
- 손해보험: 재산상의 손해를 보상하는 것이 목적입니다. 화재, 자동차사고, 해상사고 등으로 인한 재산 피해를 보장합니다.

2. 보상방식 
- 생명보험: 가입 때 약정한 보험금액을 지급받습니다. 예를 들어 사망보험금이라면 정해진 금액을 지급합니다.  
- 손해보험: 실제 발생한 손해액수만큼만 보상을 받습니다. 전부 보상되지 않을 수 있습니다.

3. 가입대상
- 생명보험: 사람의 생명과 신체를 대상으로 합니다.
- 손해보험: 재산, 자동차, 배상책임 등 다양한 위험을 담보대상으로 합니다.

4. 계약기간
- 생명보험: 일반적으로 장기계약으로 운영됩니다. 
- 손해보험: 단기계약이 주를 이룹니다. 1년 만기로 갱신하는 경우가 많습니다.

간단히 말해 생명보험은 사람의 생명과 관련된 위험을, 손해보험은 재산상의 손해 위험을 보장하는 것이 가장 큰 차이점입니다.

In [34]:
from langchain_community.document_loaders import UnstructuredAPIFileLoader
from langchain_community.document_loaders import UnstructuredFileLoader

In [92]:
from unstructured.partition.auto import partition
partition?

Signature:
partition(
    filename: Optional[str] = None,
    content_type: Optional[str] = None,
    file: Optional[IO[bytes]] = None,
    file_filename: Optional[str] = None,
    url: Optional[str] = None,
    include_page_breaks: bool = False,
    strategy: str = 'auto',
    encoding: Optional[str] = None,
    paragraph_grouper: Optional[Callable[[str], str]] = None,
    headers: Dict[str, str] = {},
    skip_infer_table_types: List[str] = ['pdf', 'jpg', 'png', 'xls', 'xlsx', 'heic'],
    ssl_verify: bool = True,
    ocr_languages: Optional[str] = None,
    languages: Optional[List[str]] = None,
    detect_language_per_element: bool = False,
    pdf_infer_table_structure: bool = False,
    extract_images_in_pdf: bool = False,
    extract_image_block_types: Optional[List[str]] = None,
    extract_image_block_output_dir: Optional[str] = None,
    extract_image_block_to_payload: bool = False,
    xml_keep_tags: bool = False,
    data_source_metadata: Optional[unstructured.documents.ele

In [43]:
loader = UnstructuredFileLoader(
    file_path="./content/sample.pdf",
    mode="elements",

    strategy="hi_res",
    hi_res_model_name="yolox",
    
    extract_images_in_pdf=True,
    pdf_infer_table_structure=False,

    extract_image_block_output_dir="./fig",
    extract_image_block_to_payload=True,

)

In [44]:
docs = loader.load()

INFO: Reading PDF for file: ./content/sample.pdf ...
INFO: Detecting page elements ...
INFO: Detecting page elements ...
INFO: Detecting page elements ...
INFO: Processing entire page OCR with tesseract...
INFO: Processing entire page OCR with tesseract...
INFO: Processing entire page OCR with tesseract...


In [64]:
docs[-5].metadata["image_base64"]

'...'

소스 코드: https://github.com/sudarshan-koirala/youtube-stuffs/blob/main/langchain/langchain_Semi_Structured_RAG.ipynb

In [ ]:
import urllib.request

url = "https://arxiv.org/pdf/2307.09288.pdf"
filename = "Llama2.pdf"
urllib.request.urlretrieve(url, filename)

In [3]:

path = "./content/"

In [53]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 2}

In [54]:

class Element(BaseModel):
    type: str
    text: Any

# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        
        print (element)
        
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

AN TIA

2 Kl 253 3| MM

x

O

[OleH o4t

EEEl )L

[=]

A A2 e |

HArE

S

AI:III}—-I de 25 2 SsHE0 WEt ol SXE foto s 4 OI'H te = AUAESE L AYLICH A4SUHES

]

O DEIIMH: 100 o2l (SEE 20| 109 0|2HY B LH0| HAZ £ US) O JIEXHaH: KOSPI200 XI4:(S28 1 Ticker: KOSPI2 index), S2It2A X4 (E2H 0 Ticker: GOLDLNPM index) O HO 2 12011 07" 042 ~2011 = 072 07 Y 16 Al O HOrCHe| 1 100 2Hel 014 100 OHRl ©F9 (28 5 D} Yol 24 Og & 2:20114 078 08Y O B J] & 2012 07 & 09 (8] 1H) O AMEQBSZ: 452 2| JIEJH 23 o EHOYHE O HZE JIFJH2: 20119 072 07 2o EXHENY HalA JIF) O 8] BIIIIA: 20123 07 € 03 2o ZIHEHY Heha JIE) 3| &9 3= 7= =3 A ESY @ SFIIEIIIIA0l FEIIFEIIAHC 100% 0|40l AL (BH)]|4 | ZHHIMUx[100%+{(2HIBIIII2H -5 % o1 A0 W IEXA JIIE) JIZ=0HA) /EIEIIZNA x 70%}] Asb (@ BEIIEIIIIA0| EZIIEINAO 100% 0I9F01 HL (2H)]% ZOHO{ 2 OH % B0l W2 IEX JIF) SHESHX100% # PED| 201 B =(0)| BOIDI A -2 2| EIHA)/E £ F0HA (S, {(SFDI - A2 ZDIEINA)/Z R EINAXT70%}I2 A CHAMA2 Olat = AD 4| EXg0 AlY 1 SRS EXNE F2 O o|atet @O @ § JIEXAS SHDIEIEIF20| KOS

In [51]:
for e in categorized_elements:
    
    print (e.type)
    print (e)
    print ("==")

text
type='text' text='이 설명서는 금융소비자의 권익 보호 및 금융상품에 대한 이해 증진을 위하여 금융상품의 핵심내용을 쉽게 이해하실 수 있도록 작성한 것입니다. 상품내용을 충분히 이해하신 다음에 청약여부를 결정하시기 바랍니다.\n\n핵심설명서  \n\n[미래에셋증권 제 253 회 파생결합증권(DLS)(원금보장형)]\n\nA\n\n1\n\n상품 개요\n\n|\n\n○ 모집가액: 100 억원 (모집된 금액이 10 억 미만일 경우 발행이 취소될 수 있음)\n\n○ 기초자산: KOSPI200 지수(블룸버그 Ticker: KOSPI2 index),\n\n금가격지수 (블룸버그 Ticker: GOLDLNPM index)\n\n○ 청약기간 : 2011 년 07 월 04 일 ~2011 년 07 월 07 일 16 시\n\n○ 청약단위 : 100 만원 이상 100 만원 단위 (발행 후 추가 납입 불가)\n\n○ 발 행 일 : 2011 년 07 월 08 일\n\n○ 만 기 일 : 2012 년 07 월 09 일(만기 1 년)\n\n○ 상품위험등급: 4 등급\n\n2 기준가격 결정일 및 평가방법\n\n○ 최초 기준가격: 2011 년 07 월 07 일의 종가(해당 거래소 기준) ○ 만기 평가가격: 2012 년 07 월 03 일의 종가(해당 거래소 기준)\n\n3|\n\n3\n\n손익 구조'
==
table
type='table' text='구 분 내 용 만기상환금액 ① 만기평가가격이 최초기준가격의 100% 이상인 경우 (만기수 만기 익률이 낮은 기초자산 기준) 기준가격) /최초기준가격 × 70%}] 상환 ② 만기평가가격이 최초기준가격의 100% 미만인 경우 (만기수 익률이 낮은 기초자산 기준) 총액면금액×100%'
==
text
type='text' text='※ 만기수익률=(만기평가가격-최초기준가격)/최초기준가격\n\n(단, {(만기평가가격-최초기준가격)/최초기준가격×70%}은 소수점 다섯째자리 이하 절사)\n\n4|\n\n4\n\n투자수익 사례 : 1 억원을 투자

## 1. Bedrock Client 생성

In [24]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [25]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [26]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [27]:
# llm_text = Bedrock(
#     model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
#     client=boto3_bedrock,
#     model_kwargs={
#         "max_tokens_to_sample": 1024
#     },
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
# )

# llm_text


llm_text = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Sonnet"),
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1024,
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7fd148434ac0>, region_name='us-west-2', model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'anthropic_version': 'bedrock-2023-05-31', 'max_tokens': 1024}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fd27afb38b0>])

In [32]:

# Apply to tables
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

요약하이 면 표는 다음과만 같기 습니다시 상환금액:에 대한 

이 내용표을 설명는하고  KOS있PI200 습니다.지수  옵션

두 가지  경우계약에 로 대한 나뉘어 만기있 는시데:나리오

1. 만기를 평가가격이 최초기보여줍니준가격의 다.100% 이상 일 

- 때:최초 기준가격 결정일 에는
상 환금KOS액 PI200 =지수 {최가 저250pt기준가격이(고,= 1최초기준가계약당 1격 ,500달러또는 를 만기평가가격지 불합니다중 낮은.

-  가격만기) / 평가일 1최초기준가에격 × 70서%}

2.는 만기평가 가KOS격이 PI최200 초기준가격지수가 의 100% 20% 상승미만일 때하:
상환금액 = 총액면금액 × 100여 %300pt가 되 고, 옵션 가

요약치가 10하면 % 상승하여 1,650만기평가가격 수준달러가 됩니에다 . 따라이 경우 7% 수익률로  상환금액700의 만계산 방원식이 달라의 지는 것세전 이익을 이 보여발주고생 합니다.

- 만기평가일 2에있서는 KOSPI200 지습니다수는 여전히 20.% 상승한 300pt이지만, 옵션 가치는 10% 하락하여 1,350달러가 됩니다. 이 경우 손실이 발생합니다.

KOSPI200 지수와 옵션 가격 간의 불일치로 인해 수익 또는 손실이 결정됩니다.

In [33]:

# Apply to texts
texts = [i.text for i in text_elements]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

이 표는 투자 상품의 수익률 구조이와  1문억원 서투자는  시 미래에셋손익증 권사례에를 서 발행하는 보여주제고 있습니다. 253회 

첫 번째 본 파생결합증문그서는래 프는 제E목LS이 권(DLS)(("수익률 원금주보장형그래프"이)가며 다에 연계양한대증권 퍼센) 한트상품 핵심와 에 대한 설명서입니비율설다명서입니, 다. 계산. 주주식요 등이 요있 내 습니다. 용은 내용하지만 은 다음과 같다음과 같습니다:습니다

- 자세모집:금액한 설명

1. :이  상품 개요100없억원: 어  (원금보장그형10억원 미만래프일  경우의 ELS 발행 취로 정확소기초자산한  은의미 KOSPI가능를 200 지수)
- 와파 악하기 기초자산금어렵습니: KOSPI가다격지수200 지수입니.다, 금. 가격 

2. 수

두 번째 부분은 익 지구수조: 1억원만기  
- 청약을 투자시기간했을  때의: 2두 011 년 기사례초자7산월 4일를 의 가격 설명변동에 합니다. KOSPI200 ~ 따라  지수수익이 가 결정됩니250pt다,.  금가격지수7금 가격  지수가 기일
-$ 청1,500준약단위: 일 때으로 손100만원 이를 익상이 산정되며, 최기준으로 대  100만원 70%까지 수익하이 고단가능 있습니다위
- 발행일: 2011년 7월 8일. 합니  다
- 만하지만 기일: 2구.

3.012 년 7월체적인 투 9일자 (만기 1년) 
- 손익 위험등급:금액 4등중도급이
- 나 기준가격 비결정율은 : 제시되상환: 최초 지 않2011년 7투자자았습니다월 7일  종가,요 만기 .2012년 7청

전월  시 반적으로 3일 종가중도
-투상 자 환원금보장상이품의  가수능익하 나, 구조6개월 이와 내에초형 는 기 조상건을 공품개괄적으로 설명하고 있지

투자정만가액의 자90% 이는,  상세상품, 부 6개월내용을 이 잘 이해한후 후  에는 청내약용95% 이상 을 이나 여부를 결지계정산 방급하합니라고다 안내하고 있습니다.식.에 

4. 대한  위험추가 정보요가 인: 필요원금손실 해 보입니가능성다, .발행회사 신용위험, 기초

In [34]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [35]:
# Prompt
prompt_text="""
\n\nHuman:
You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text.
Table or text chunk: {element}
\n\nAssistant:
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

summarize_chain = {"element": lambda x:x} | prompt | llm_text | StrOutputParser()

In [36]:
prompt

ChatPromptTemplate(input_variables=['element'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['element'], template='\n\n\nHuman:\nYou are an assistant tasked with summarizing tables and text. Give a concise summary of the table or text.\nTable or text chunk: {element}\n\n\nAssistant:\n'))])

In [43]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=llm_emb
)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [Document(page_content=s,metadata={id_key: doc_ids[i]}) for i, s in enumerate(text_summaries)]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [Document(page_content=s,metadata={id_key: table_ids[i]}) for i, s in enumerate(table_summaries)]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [47]:
summary_tables

[Document(page_content='이 표는 만기 시 상환금액에 대한 내용을 설명하고 있습니다. \n\n두 가지 경우로 나뉘어 있는데:\n\n1. 만기평가가격이 최초기준가격의 100% 이상일 때: \n상환금액 = {최저기준가격(=최초기준가격 또는 만기평가가격 중 낮은 가격) / 최초기준가격 × 70%}\n\n2. 만기평가가격이 최초기준가격의 100% 미만일 때:\n상환금액 = 총액면금액 × 100% \n\n요약하면 만기평가가격 수준에 따라 상환금액의 계산 방식이 달라지는 것을 보여주고 있습니다.', metadata={'doc_id': '5a9b9725-bef1-4f10-83e9-d57fb6afe05c'}),
 Document(page_content='요약하면 다음과 같습니다:\n\n이 표는 KOSPI200 지수 옵션 계약에 대한 만기 시나리오를 보여줍니다. \n\n- 최초 기준가격 결정일에는 KOSPI200 지수가 250pt이고, 1계약당 1,500달러를 지불합니다.\n\n- 만기평가일 1에서는 KOSPI200 지수가 20% 상승하여 300pt가 되고, 옵션 가치가 10% 상승하여 1,650달러가 됩니다. 이 경우 7% 수익률로 700만원의 세전 이익이 발생합니다.\n\n- 만기평가일 2에서는 KOSPI200 지수는 여전히 20% 상승한 300pt이지만, 옵션 가치는 10% 하락하여 1,350달러가 됩니다. 이 경우 손실이 발생합니다.\n\nKOSPI200 지수와 옵션 가격 간의 불일치로 인해 수익 또는 손실이 결정됩니다.', metadata={'doc_id': 'f97f5295-c003-430c-a77a-b15b270f9a72'})]

In [45]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

# Prompt template
template = """
\n\nHuman:
Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
\n\nAssistant:
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM

#model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm_text
    | StrOutputParser()
)

In [46]:
chain.invoke("KOSPI200 가격은?")

주어진 정보를 바탕으로 KOSPI200 지수의 가격은 다음과 같습니다.

- 최초기준가격결정일(2011년 7월 7일) KOSPI200 종가: 250pt
- 만기평가일 1의 KOSPI200 종가: 300pt (최초 대비 20% 상승)  
- 만기평가일 2의 KOSPI200 종가: 300pt (최초 대비 20% 상승)

따라서 최초기준가격결정일의 KOSPI200 지수는 250pt이었고, 두 번의 만기평가일에는 모두 300pt로 20% 상승했음을 알 수 있습니다.

'주어진 정보를 바탕으로 KOSPI200 지수의 가격은 다음과 같습니다.\n\n- 최초기준가격결정일(2011년 7월 7일) KOSPI200 종가: 250pt\n- 만기평가일 1의 KOSPI200 종가: 300pt (최초 대비 20% 상승)  \n- 만기평가일 2의 KOSPI200 종가: 300pt (최초 대비 20% 상승)\n\n따라서 최초기준가격결정일의 KOSPI200 지수는 250pt이었고, 두 번의 만기평가일에는 모두 300pt로 20% 상승했음을 알 수 있습니다.'

In [ ]:

prompt1 = "나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."
messages = [
    HumanMessage(content=prompt1)
]

# messages = [
#     {"role": "user", "content": [{"type": "text", "text": prompt1}]},
# ]

response1 = llm_text.invoke(messages)

### Embedding 모델 선택

In [39]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [40]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [42]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [ ]:
import boto3
from utils.ssm import parameter_store

In [ ]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [ ]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [ ]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [ ]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

### OpenSearch Client 생성

In [ ]:
from utils.opensearch import opensearch_utils

In [ ]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 4. Retriever based on Hybrid Search 정의
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [ ]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [ ]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    index_name=index_name,
    llm_emb=llm_emb,
    llm_text=llm_text,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [ ]:
from utils.rag import show_context_used

In [ ]:
#query = "중지된 경우 이체"
query = "vidio max size?"

In [ ]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

- update parameters

In [ ]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=0,
    filter=[],
    #filter=[
    #    {"term": {"metadata.seq_num": "64"}},
    #],
    async_mode=True
)

In [ ]:
#query = "중지된 경우 이체"
query = "vidio max size?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

## 5. RAG using RetrievalQA powered by LangChain

In [ ]:
from utils.rag import prompt_repo
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [ ]:
PROMPT = prompt_repo.get_qa(prompt_type="answer_only") # ["answer_only", "answer_with_ref"]
pprint (PROMPT.template)

### Update Search Params (Optional)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=3,
    #filter=[
    #    {"term": {"metadata.project": "KS"}},
    #],
    filter=[],
    async_mode=True
)

### Request

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [ ]:
#query = "중지된 경우 이체"
query = "vidio max size?"
response = qa(query)

In [ ]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(response['source_documents'])